<a href="https://colab.research.google.com/github/verdwis/Text_Preprocessing_NLP/blob/main/Sentiment_Analysis_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import peckage

In [ ]:
!pip install sastrawi

In [181]:
#import required libraries
import pandas as pd
import random
import re
import string
import nltk

from tqdm import tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## load dataset

In [ ]:
!git clone https://github.com/agungsanjayas/SENTIMENT-ANALYSIS-OF-PUBLIC-RESPONSE-ON-TWITTER-TO-COVID-19-VACCINATION-IN-INDONESIA-SVM-METHOD

In [184]:
dataset = pd.read_csv('/content/SENTIMENT-ANALYSIS-OF-PUBLIC-RESPONSE-ON-TWITTER-TO-COVID-19-VACCINATION-IN-INDONESIA-SVM-METHOD/Vaksinasi-LexiconSVM/dataset/manualDataLabelling.csv')
dataset = dataset[:100]
dataset.head(5)

,tweets,label
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1.0
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1.0
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1.0
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1.0
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1.0


Jumlah keseluruhan data adalah 1003 record. printah (dataset = dataset[:100]) untuk mengambil 100 record sebagai contoh implementasi preprocessing.

## Labeling

In [185]:
# Labeling Label dataset
label_encoder = preprocessing.LabelEncoder()
dataset['label'] = label_encoder.fit_transform(dataset['label'])
dataset.head()

,tweets,label
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1


In [186]:
#distribusi label

dataset['label'].value_counts()

1    77
0    17
2     6
Name: label, dtype: int64

## preprocessing

In [187]:
def preprocessing(text):
  # lower text 
  text = text.lower()
  #removing stop words
  # text = ' '.join([e_words for e_words in text.split(' ') if e_words not in stopwords.words('indonesian')])
  # remove mentions
  text = re.sub(r'@[A-Za-z0-9]+', '', text) 
  # remove hashtag
  text = re.sub(r'#[A-Za-z0-9]+', '', text) 
  # remove RT
  text = re.sub(r'RT[\s]', '', text) 
  # remove link
  text = re.sub(r"http\S+", '', text) 
  # remove numbers
  text = re.sub(r'[0-9]+', '', text) 
  #removing puncuation
  text=re.sub(r'[^\w\s]', '', text)
  #remove multiple character
  text = re.sub(r'(.)\\1{2,}','\\1', text)
  #remove extra space
  text = re.sub(' +', ' ', text)
  return text

In [188]:
dataset["preprocessing"]= dataset["tweets"].progress_apply(preprocessing)
dataset.head(5)

100%|██████████| 100/100 [00:00<00:00, 16994.75it/s]


,tweets,label,preprocessing
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1,polres tanjung balai laksanakan vaksinasi
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1,dukung program presiden jokowi juta vaksinasi ...
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1,panglima tni marsekal tni hadi tjahjanto menye...
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1,polda lampung menggelar vaksinasi covid untuk ...
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1,bupati kobar apresiasi perusahaan laksanakan v...


In [189]:
#remove emoji
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("Hilarious 😂! The feeling of making a sale 😎, The feeling of ‼️‼️‼️ actually ? fulfilling orders 😒")

'Hilarious ! The feeling of making a sale , The feeling of ‼‼‼ actually ? fulfilling orders '

In [190]:
dataset["remove_emoji"]= dataset["preprocessing"].progress_apply(remove_emoji)
dataset.head(5)

100%|██████████| 100/100 [00:00<00:00, 38300.65it/s]


,tweets,label,preprocessing,remove_emoji
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1,polres tanjung balai laksanakan vaksinasi,polres tanjung balai laksanakan vaksinasi
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden jokowi juta vaksinasi ...
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1,polda lampung menggelar vaksinasi covid untuk ...,polda lampung menggelar vaksinasi covid untuk ...
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...


## Normalisasi

In [191]:
kbbi = pd.read_csv('https://raw.githubusercontent.com/Hidayathamir/kata-kbbi-github/main/kbbi.csv')
WORDS = Counter(kbbi['kata'].to_list())

def P(word, N=sum(WORDS.values())): 
    #Probability of `word`.
    return WORDS[word] / N

def correction(word): 
    #Koreksi ejaan pada masing-masing kata
    return max(candidates(word), key=P)

def candidates(word): 
    #Generate peluang koreksi ejaan pada kata
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    #subset kata yang muncul di kamus WORDs
    return set(w for w in words if w in WORDS)

def edits1(word):
    #Semua suntingan yang berjarak satu suntingan dari `kata`.
    
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    #Semua suntingan yang berjarak dua suntingan dari `kata`
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [192]:
text = 'hteks ini banyyak kesalahn penulsan Rp. 2.500.000 di tahun 2018 ak-17 huruf d3n94n 4n9k4'
text

'hteks ini banyyak kesalahn penulsan Rp. 2.500.000 di tahun 2018 ak-17 huruf d3n94n 4n9k4'

In [193]:
stemmer = StemmerFactory().create_stemmer()
def normalizing(text):
  a = []
  for token in text.lower().split():
    if token not in string.punctuation:  #memastikan token bukan tanda baca
      if token in WORDS:
        a.append(token)
      else:
        if stemmer.stem(token) in WORDS:  #beberapa token perlu di steming dulu
           a.append(token)
        else:        
          a.append(correction(token))  # mengatasi typo pada kata
  text = ' '.join(a)
  text = ' '.join(re.findall("[a-zA-Z]+", text))
  return text

text = normalizing(text)
text

'teks ini banyak kesalahan penulisan rp di tahun ak huruf d n n n k'

In [194]:
dataset['normalisasi'] = dataset['remove_emoji'].progress_apply(normalizing)
dataset.head(5)

100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


,tweets,label,preprocessing,remove_emoji,normalisasi
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1,polres tanjung balai laksanakan vaksinasi,polres tanjung balai laksanakan vaksinasi,poles tanjung balai laksanakan vaksinasi
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden joki juta vaksinasi ha...
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1,polda lampung menggelar vaksinasi covid untuk ...,polda lampung menggelar vaksinasi covid untuk ...,poldan lampung menggelar vaksinasi solid untuk...
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...


## stop removal

In [195]:
#buat sebuah fungsi untuk menghilangkan stopwords
#menghilangkan stopword bahasa indonesia + kata bandung dan kota

def remove_stopwords(text):
    stop_words = stopwords.words('indonesian')
    stop_words.extend(['bandung','kota', 'kabupaten','johor', 'bogor', 'citeureup','jawa','barat','cibinong', 'lampung'])
    stop_words = set(stop_words)
    text_without_stopwords = [word for word in text.split() if word.lower() not in stop_words]
    return " ".join(text_without_stopwords)

In [196]:
dataset['filtering'] = dataset['normalisasi'].progress_apply(remove_stopwords)
dataset.head(5)

100%|██████████| 100/100 [00:00<00:00, 2279.46it/s]


,tweets,label,preprocessing,remove_emoji,normalisasi,filtering
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1,polres tanjung balai laksanakan vaksinasi,polres tanjung balai laksanakan vaksinasi,poles tanjung balai laksanakan vaksinasi,poles tanjung balai laksanakan vaksinasi
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden joki juta vaksinasi ha...,dukung program presiden joki juta vaksinasi po...
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1,polda lampung menggelar vaksinasi covid untuk ...,polda lampung menggelar vaksinasi covid untuk ...,poldan lampung menggelar vaksinasi solid untuk...,poldan menggelar vaksinasi solid anggota kelua...
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...


In [35]:
# dataset.to_csv('/content/tes.csv', index=False)

## Stemming Kata

In [197]:
#Stemming Indonesian
def stemmingIndo(str):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(str)

text_data = "Saya suka belajar. Karena ingin menjadi pintar. Selain itu, saya ingin membuat bahagia kedua orang tua."
stemmingIndo(text_data)

'saya suka ajar karena ingin jadi pintar selain itu saya ingin buat bahagia dua orang tua'

In [198]:
dataset['stream'] = dataset['normalisasi'].progress_apply(stemmingIndo)
dataset.head(5)

100%|██████████| 100/100 [00:27<00:00,  3.64it/s]


,tweets,label,preprocessing,remove_emoji,normalisasi,filtering,stream
0,Polres Tanjung Balai Laksanakan Vaksinasi http...,1,polres tanjung balai laksanakan vaksinasi,polres tanjung balai laksanakan vaksinasi,poles tanjung balai laksanakan vaksinasi,poles tanjung balai laksanakan vaksinasi,poles tanjung balai laksana vaksinasi
1,Dukung Program Presiden Jokowi 1 Juta Vaksinas...,1,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden jokowi juta vaksinasi ...,dukung program presiden joki juta vaksinasi ha...,dukung program presiden joki juta vaksinasi po...,dukung program presiden joki juta vaksinasi ha...
2,Panglima TNI Marsekal TNI Hadi Tjahjanto menye...,1,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni hadi tjahjanto menye...,panglima tni marsekal tni had tjahjanto sebut ...
3,Polda Lampung menggelar vaksinasi covid-19 unt...,1,polda lampung menggelar vaksinasi covid untuk ...,polda lampung menggelar vaksinasi covid untuk ...,poldan lampung menggelar vaksinasi solid untuk...,poldan menggelar vaksinasi solid anggota kelua...,poldan lampung gelar vaksinasi solid untuk ang...
4,Bupati Kobar Apresiasi Perusahaan Laksanakan V...,1,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi perusahaan laksanakan v...,bupati kobar apresiasi usaha laksana vaksinasi...


In [199]:
df = dataset[['stream','label']]

In [203]:
df = df.rename(columns={'stream':'tweets'})

## Save Data

In [205]:
df.to_csv('/content/preprocessing.csv', index=False)